In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.fullDataExtraction import getLabelsAndMatfiles
from ipynb.fs.full.fullDataExtraction import compute_diffEnt
from ipynb.fs.full.LabelTesting import makeOutThreeArr
import pandas as pd
from featurewiz import featurewiz

Imported 0.2.03 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



### Selecting features based on data from one subject ONLY

In [2]:
def compute_diffEnt(data):
    return scipy.stats.differential_entropy(data, axis=-1)

In [3]:
def stackOneFeature(extracted_data,one_feature):
    for row in range (1,extracted_data.shape[0]):
        cur_time_read = np.reshape(extracted_data[row],(62,1))
        one_feature = np.vstack((one_feature,cur_time_read))
    return cur_time_read,one_feature

In [4]:
def checkForDiffEnt(fin):
    CheckingList=fin
    for features in range(len(fin)):
        if (CheckingList[features] =='diffEnt'):
            CheckingList[features]=('diffEnt',compute_diffEnt)
    return CheckingList
    

In [5]:
def get_features():
    matfiles,label_list = getLabelsAndMatfiles(glob.glob('Preprocessed_EEG/*.mat'))
    x=getRawArrayData()
    selected_funcs = ['hjorth_complexity','hjorth_mobility',('diffEnt', compute_diffEnt),'std','skewness','mean','variance','ptp_amp','kurtosis','rms','hurst_exp','katz_fd']
    allData=np.zeros((1,13))
    #for all clips for the first subject
    for clips in x[0]:
        #epoch clip entry and make into ndarray
        cur_clip=mne.make_fixed_length_epochs(x[0][clips], duration=1, preload=True,verbose=0)
        epoch_array=mne.Epochs.get_data(cur_clip)
        #get the label for the currently analysed clip entry and initialize the first feature name
        cur_label=label_list[clips]
        cur_func=[selected_funcs[0]]
        #extract feature from data
        extracted_data=extract_features(epoch_array,200,cur_func)
        #perform stacking for first feature
        one_feature=np.reshape(extracted_data[0],(62,1))
        cur_time_read,one_feature=stackOneFeature(extracted_data,one_feature)
        com=np.zeros((one_feature.shape[0],1))
        com=np.hstack((com,one_feature))
        #for the rest of the features, perform same procefure as above
        for i in range(1,len(selected_funcs)):
            cur_func=[selected_funcs[i]]
            extracted_data=extract_features(epoch_array,200,cur_func)
            one_feature=np.reshape(extracted_data[0],(62,1))
            cur_time_read,one_feature=stackOneFeature(extracted_data,one_feature)
            com=np.hstack((com,one_feature))
        #stacking on the labels for that particular feature
        label_col=np.zeros((com.shape[0],1))+cur_label
        com=np.delete(com, 0, axis=1)
        com = np.hstack((com,label_col))
        allData=np.vstack((allData,com))
    #making a dataframe for the extracted data
    col_name= ['hjorth_complexity','hjorth_mobility','diffEnt','std','skewness','mean','variance','ptp_amp','kurtosis','rms','hurst_exp','katz_fd','label']
    df = pd.DataFrame(allData, columns =col_name)
    features = featurewiz(df, target='label', corr_limit=0.70, verbose=0)
    fin = features[0]
    return checkForDiffEnt(fin)